# After creation of identified csv from collaboration and citation approach, We will merge them into one.

In [4]:
import ast
import pandas as pd
import vaex
import os.path
from ast import literal_eval
from pathlib import Path

FILES = ['compbiology', 'biology', 'medicine', 'genetics', 'ntds', 'pathogenes', 'plosone', 'srep']

# Path to data
PATH_SAVED_CSV = '../../data-computed-MAG'
PATH_MAG = '../../data-MAG/'

# Path to data to save
PATH_SAVE = '../../data-computed-MAG'
Path(PATH_SAVE).mkdir(parents=True, exist_ok=True) # create save dir if not exists

In [5]:
def see(var):
    """funtion to check the contents of variable (dict or set)"""
    print(type(var))
    if type(var) == dict:
        for i, (k, v) in enumerate(var.items()):
            print(k, ":", v)
            if i == 4:
                break
    if type(var) == set:
        print(list(var)[:5])
        
        
def read_vaex(file_path):
    """
    It may take times to read files for the first time.
    From second time, the loading time will be shorter.
    """
    if os.path.exists(file_path):
        print('file exists')
        return vaex.open(file_path+'.hdf5')
    return vaex.from_csv(file_path, convert=True, chunk_size=10_000_000)

# Read MAG ID result

In [6]:
# result of collaboration identifier
df_determinedAcknow_oneId = pd.read_csv(f'{PATH_SAVED_CSV}/d1Collaboration_result.csv')

# result of citation identifier
df_result_cited = pd.read_csv(f'{PATH_SAVED_CSV}/references_result.csv')

In [4]:
len(df_result_cited), len(df_determinedAcknow_oneId)

(245508, 797588)

In [5]:
# merge them
df_acknowId_d1collab_ref_merged = pd.merge(df_determinedAcknow_oneId, df_result_cited,  
          how='outer', on=['pid', 'acknow'])
# drop duplicated row
df_acknowId_d1collab_ref_merged = df_acknowId_d1collab_ref_merged.drop_duplicates(subset=['pid', 'acknow'])
df_acknowId_d1collab_ref_merged.head()

,pid,acknow,DetermindedAcknowId,ReferencedAuthorIds
0,2145218186,Aviv Regev,1.893730e+09,NaN
2,2105003357,Aviv Regev,1.893730e+09,NaN
4,2168608639,Aviv Regev,1.893730e+09,NaN
12,2033343333,Aviv Regev,1.893730e+09,3.147624e+09
15,2105898083,Aviv Regev,1.893730e+09,NaN


# Divide into several type of dfs and take IDs of acknowledged scholars

In [6]:
# Acknoweldged scholars identified only by collaboration approach
df_acknowId_from_d1Collab = df_acknowId_d1collab_ref_merged[df_acknowId_d1collab_ref_merged['ReferencedAuthorIds'].isnull()]
# Acknoweldged scholars identified only by citation approach
df_acknowId_from_ref = df_acknowId_d1collab_ref_merged[df_acknowId_d1collab_ref_merged['DetermindedAcknowId'].isnull()]
# Acknoweldged scholars identified by both collaboration and citation approach
df_acknowId_from_both = df_acknowId_d1collab_ref_merged[df_acknowId_d1collab_ref_merged['DetermindedAcknowId'] == df_acknowId_d1collab_ref_merged['ReferencedAuthorIds']]

# IDs of acknowledged scholars
df_acknowId = pd.concat([df_acknowId_from_d1Collab, df_acknowId_from_ref, df_acknowId_from_both])

# make new acknowId col
df_acknowId['acknowId'] = [ int(row['DetermindedAcknowId']) if np.isnan(row['ReferencedAuthorIds']) else (row['ReferencedAuthorIds']) for i, row in df_acknowId.iterrows()]
df_acknowId['acknowId'] = df_acknowId['acknowId'].astype(int)
df_acknowId.head()

,pid,acknow,DetermindedAcknowId,ReferencedAuthorIds,acknowId
0,2145218186,Aviv Regev,1.893730e+09,NaN,1893730172
2,2105003357,Aviv Regev,1.893730e+09,NaN,1893730172
4,2168608639,Aviv Regev,1.893730e+09,NaN,1893730172
15,2105898083,Aviv Regev,1.893730e+09,NaN,1893730172
28,2145218186,Dalit May,2.800119e+09,NaN,2800119218


In [7]:
len(df_acknowId)

235566

In [8]:
len(df_acknowId) # after

235566

#  Create Edgelist

### Merge with authors info of papers

In [8]:
# read paperAuthorAffiliation of MAG
vaex_mag_pauthorAffil = read_vaex(f'{PATH_MAG}/PaperAuthorAffiliations/PaperAuthorAffiliations.csv')

file exists


In [10]:
# summarise authors information of the acknowledged papers
vaex_row_paperinfo = vaex_mag_pauthorAffil[vaex_mag_pauthorAffil.PaperId.isin(df_acknowId.pid.tolist())]

# gropuby paper
df_row_paperinfo = vaex_row_paperinfo.to_pandas_df().rename({' AuthorId': 'AuthorId'}, axis=1)
df_row_paperinfo_groupby_paper = df_row_paperinfo.groupby('PaperId')['AuthorId'].apply(list).reset_index()
df_row_paperinfo_groupby_paper.head()

,PaperId,AuthorId
0,27879997,"[1975038829, 1975038829, 1975038829, 205481105..."
1,30683393,"[2020286370, 2030482327, 2118626585, 211862658..."
2,33210814,"[2064773457, 2194711037]"
3,36655823,"[2126235715, 2328750602, 2507239260, 260555830..."
4,51943945,"[2031533187, 2031533187, 2469105369, 246910536..."


In [11]:
df_acknow_wCollaborater = pd.merge(df_acknowId, df_row_paperinfo_groupby_paper,  
          how='inner', left_on='pid', right_on='PaperId')

df_acknow_wCollaborater.head()

,pid,acknow,DetermindedAcknowId,ReferencedAuthorIds,acknowId,PaperId,AuthorId
0,2145218186,Aviv Regev,1.893730e+09,NaN,1893730172,2145218186,"[1219376531, 2079978182, 2125555973]"
1,2145218186,Dalit May,2.800119e+09,NaN,2800119218,2145218186,"[1219376531, 2079978182, 2125555973]"
2,2145218186,Ruth Hershberg,2.155149e+09,NaN,2155149074,2145218186,"[1219376531, 2079978182, 2125555973]"
3,2145218186,Yael Altuvia,2.212855e+08,NaN,221285518,2145218186,"[1219376531, 2079978182, 2125555973]"
4,2145218186,Lena Nekludova,NaN,1.991418e+09,1991417625,2145218186,"[1219376531, 2079978182, 2125555973]"


### drop duplication of author ID

In [12]:
df_acknow_wCollaborater['AuthorId'] = df_acknow_wCollaborater['AuthorId'].apply(set).apply(list)

### drop the cases the acknowledging author is the same as the acknoweldged author

In [67]:
%%time
# drop the cases the acknowledging author is the same as the acknoweldged author
df_acknow_wCollaborater['acknowIsAuthor'] = df_acknow_wCollaborater.apply(lambda x: True if x['acknowId'] in x['AuthorId'] else False, axis=1) 
df_acknow_notAuthor = df_acknow_wCollaborater[~df_acknow_wCollaborater['acknowIsAuthor']]
df_acknow_notAuthor

CPU times: user 1.15 s, sys: 21.9 ms, total: 1.18 s
Wall time: 1.17 s


,pid,acknow,DetermindedAcknowId,ReferencedAuthorIds,acknowId,PaperId,AuthorId,acknowIsAuthor
0,2145218186,Aviv Regev,1.893730e+09,NaN,1893730172,2145218186,"[1219376531, 2125555973, 2079978182]",False
1,2145218186,Dalit May,2.800119e+09,NaN,2800119218,2145218186,"[1219376531, 2125555973, 2079978182]",False
2,2145218186,Ruth Hershberg,2.155149e+09,NaN,2155149074,2145218186,"[1219376531, 2125555973, 2079978182]",False
3,2145218186,Yael Altuvia,2.212855e+08,NaN,221285518,2145218186,"[1219376531, 2125555973, 2079978182]",False
4,2145218186,Lena Nekludova,NaN,1.991418e+09,1991417625,2145218186,"[1219376531, 2125555973, 2079978182]",False
...,...,...,...,...,...,...,...,...
235561,3118328364,Zhaoyang Wang,2.596801e+09,2.596801e+09,2596801132,3118328364,"[2057551680, 140022714, 2120368504, 2151741246]",False
235562,3118331071,Masayoshi Nishiyama,2.187045e+09,2.187045e+09,2187044629,3118331071,"[3118709908, 1887262164, 3118984329, 312077809...",False
235563,3127263314,Andrea Berton,1.974124e+09,1.974124e+09,1974123948,3127263314,"[2008829488, 2557919313]",False
235564,3127584525,Hans Keppler,2.099200e+09,2.099200e+09,2099200267,3127584525,"[2012994082, 1945867165, 2398974053]",False


# format

In [68]:
edgelist = df_acknow_notAuthor[['PaperId', 'acknowId', 'AuthorId']].rename({'acknowId': 'AcknowId'}, axis=1)
edgelist.head()

,PaperId,AcknowId,AuthorId
0,2145218186,1893730172,"[1219376531, 2125555973, 2079978182]"
1,2145218186,2800119218,"[1219376531, 2125555973, 2079978182]"
2,2145218186,2155149074,"[1219376531, 2125555973, 2079978182]"
3,2145218186,221285518,"[1219376531, 2125555973, 2079978182]"
4,2145218186,1991417625,"[1219376531, 2125555973, 2079978182]"


### Here, we try once to create 'author-acknoweldged scholar' edgelist, so we did some extra treatment.

In [69]:
edgelist['weight'] = edgelist['AuthorId'].apply(lambda x: 1/len(set(x)))
edgelist.head()

,PaperId,AcknowId,AuthorId,weight
0,2145218186,1893730172,"[1219376531, 2125555973, 2079978182]",0.333333
1,2145218186,2800119218,"[1219376531, 2125555973, 2079978182]",0.333333
2,2145218186,2155149074,"[1219376531, 2125555973, 2079978182]",0.333333
3,2145218186,221285518,"[1219376531, 2125555973, 2079978182]",0.333333
4,2145218186,1991417625,"[1219376531, 2125555973, 2079978182]",0.333333


### save as  csv

In [71]:
edgelist.drop_duplicates(subset=['PaperId', 'AcknowId']).explode('AuthorId').to_csv("../../data-computed-MAG/edgelist.csv", index=False)

In [79]:
edgelist[edgelist['AcknowId'] == 2155683674].groupby('PaperId')['weight'].sum()

PaperId
1969127928    0.076923
1976788601    0.250000
1986437762    0.111111
2006954859    0.200000
2009680828    0.125000
2018690457    0.100000
2021209057    0.250000
2036469855    0.125000
2041875417    0.071429
2046781790    0.166667
2055907168    0.333333
2056184324    0.250000
2064481342    0.111111
2089615515    0.100000
2098194107    0.052632
2125242560    0.200000
2169923259    0.052632
2406315375    0.200000
2517801085    0.062500
2611282009    0.052632
2747040076    0.111111
2888143680    0.058824
3047065714    0.058824
Name: weight, dtype: float64

### check wieght

In [41]:
a = edgelist

In [35]:
a['weight2'] = a['AuthorId'].apply(lambda x: 1/len(set(x)))

In [36]:
a['weight_diff'] = a['weight'] - a['weight2']

In [37]:
a['len_auth'] = a['AuthorId'].apply(lambda x: len(set(x)))

In [38]:
a['len_weight'] = a['weight'] * a['len_auth']

In [39]:
a

,PaperId,AcknowId,AuthorId,weight,weight2,weight_diff,len_auth,len_weight
0,2145218186,1893730172,"[1219376531, 2079978182, 2125555973]",0.333333,0.333333,0.0,3,1.0
1,2145218186,2800119218,"[1219376531, 2079978182, 2125555973]",0.333333,0.333333,0.0,3,1.0
2,2145218186,2155149074,"[1219376531, 2079978182, 2125555973]",0.333333,0.333333,0.0,3,1.0
3,2145218186,221285518,"[1219376531, 2079978182, 2125555973]",0.333333,0.333333,0.0,3,1.0
4,2145218186,1991417625,"[1219376531, 2079978182, 2125555973]",0.333333,0.333333,0.0,3,1.0
...,...,...,...,...,...,...,...,...
235561,3118328364,2596801132,"[140022714, 2057551680, 2120368504, 2151741246]",0.250000,0.250000,0.0,4,1.0
235562,3118331071,2187044629,"[1887262164, 2188451550, 2188451550, 247605287...",0.142857,0.142857,0.0,7,1.0
235563,3127263314,1974123948,"[2008829488, 2557919313]",0.500000,0.500000,0.0,2,1.0
235564,3127584525,2099200267,"[1945867165, 2012994082, 2398974053]",0.333333,0.333333,0.0,3,1.0


In [88]:
a[a['len_weight'] != 1]

,PaperId,AcknowId,AuthorId,weight,weight2,weight_diff,len_auth,len_weight
4912,2488914148,2971766166,"[5665757, 120063865, 202064119, 215276197, 220...",0.005348,0.005348,0.0,187,1.0
4913,2488914148,2013623313,"[5665757, 120063865, 202064119, 215276197, 220...",0.005348,0.005348,0.0,187,1.0
4914,2488914148,1082027422,"[5665757, 120063865, 202064119, 215276197, 220...",0.005348,0.005348,0.0,187,1.0
4915,2488914148,2080042687,"[5665757, 120063865, 202064119, 215276197, 220...",0.005348,0.005348,0.0,187,1.0
4916,2488914148,2088249918,"[5665757, 120063865, 202064119, 215276197, 220...",0.005348,0.005348,0.0,187,1.0
...,...,...,...,...,...,...,...,...
79723,2855009822,923897920,"[297100734, 731780635, 835450655, 1213735215, ...",0.020408,0.020408,0.0,49,1.0
79724,2855009822,2226890588,"[297100734, 731780635, 835450655, 1213735215, ...",0.020408,0.020408,0.0,49,1.0
79725,2855009822,2089238932,"[297100734, 731780635, 835450655, 1213735215, ...",0.020408,0.020408,0.0,49,1.0
203265,2753503144,2680912847,"[16877013, 42462615, 48407866, 1805018059, 195...",0.020408,0.020408,0.0,49,1.0


In [40]:
a.loc[4912]['len_weight']

0.9999999999999999

In [7]:
df_acknowId.head()

,pid,acknow,DetermindedAcknowId,ReferencedAuthorIds,acknowId
0,2145218186,Aviv Regev,1.893730e+09,NaN,1893730172
2,2105003357,Aviv Regev,1.893730e+09,NaN,1893730172
4,2168608639,Aviv Regev,1.893730e+09,NaN,1893730172
15,2105898083,Aviv Regev,1.893730e+09,NaN,1893730172
28,2145218186,Dalit May,2.800119e+09,NaN,2800119218
